# Structuring OCR'ed Text as Data
## 2022 Duke University Humanities Bootcamp Edition 

<div class="alert alert-block alert-info">
    <strong>Learning Objectives:</strong>
    <p>By the end of this module, you should be able to</p>
    <ul>
        <li>explain reasons for creating structured data from OCR'ed text and describe possible use cases;</li>
        <li>create structured data from OCR'ed text.</li>
    </ul>
</div>

## Table of Contents

- [Structuring OCR'ed Text as Data](#text-as-data)
- [Resources](#resources)

In this module, we'll look at ways to structure OCR'ed text as data and consider why we might want to create structured data. We'll end by looking at a sample of the structured data from the *On The Books* corpus of Jim Crow laws. This will prepare us for the [final module](06-ExploratoryAnalysis.ipynb) in which we'll be using Python and other tools to perform exploratory analyses with the *On The Books* corpus dataset.

## Structuring OCR'ed Text as Data <a class="anchor" id="text-as-data"></a>

In the [next module](06-ExploratoryAnalysis.ipynb) we'll begin exploring OCR'ed text as data. We'll look at ways to analyze text content and view laws temporally and spatially. We'll be working with the <strong><a href="https://pandas.pydata.org/" target="blank">Pandas</a></strong> library to prepare data for analysis. Pandas is an extremely useful library for working with data, and we'll just be skimming the surface of what it can do here. **However, before we can analyze text data, we need to structure it.**

We'll begin by looking at the laws, which have been stored together in one file, `on_the_books_text_jc_all_v2.txt`, which you'll find <a href="https://cdr.lib.unc.edu/concern/data_sets/7d2793593?locale=en" target="blank">here</a>. (Click Download on the webpage that loads, or click [here](https://cdr.lib.unc.edu/downloads/6q182r84s?locale=en).)

*What do you notice about how the laws have been structured in this one file? How are they separated from one another? What other kinds of data accompany them? What kinds of data do NOT accompany them?*

<img src="images/09-data-02.jpeg" width="90%" style="margin:20px; box-shadow: 25px 25px 20px -30px rgba(0, 0, 0);" alt="An excerpt from the corpus of North Carolina Jim Crow laws stored in a text file." title="An excerpt from the corpus of North Carolina Jim Crow laws stored in a text file." />

The above image shows an excerpt of the corpus file including 3 laws. Each law and its accompanying metadata (data about the law) are separated from the previous and next laws by 3 empty lines, which Python reads as 4 newline characters, `\n\n\n\n`.

Remember that in *On The Books*, Jim Crow laws were identified at the level of a chapter *section*, so each of the laws we are viewing here come from a different chapter and/or volume. The first line includes 
- the volume, listed by year(s); 
- whether the law is considered "public" (public laws focus on public institutions and public property) or "private" law (private laws regulate individual private property, private companies, etc.);
- the chapter number;
- the section number.

In the screen capture above, each law has been identified as (`Identified by:`) a Jim Crow law by a human expert or by the algorithms used by the *On The Books* team ("model"). 

<div class="alert alert-block alert-warning">
    <p>The model was developed by feeding a dataset of laws identified by human experts as either being Jim Crow or not being Jim Crow. Two human experts went through a selection of laws to make these identifications, and they did not always agree. However, computers require clear forms of classification (e.g. yes/no). The team's programmers then created a model that could "read" the selected laws ("training set") and their classifications. As the computer "read" it "learned" how laws labeled "yes" (Jim Crow) were structured and the kinds of words appear in those laws--and it did the same with "no" (not Jim Crow) laws. It used this information to go through the entire corpus and attempt to correctly identify all of the laws that human experts had not reviewed. There is more to this process, and if you wish to learn more, you'll find the documentation and code in <a href="https://github.com/UNC-Libraries-data/OnTheBooks" target="blank">the <em>On The Books</em> Github repository</a>.</p>
    </div>

After the `Identified by:` field, the metadata accompanying the law then includes the full name of the chapter followed by the full text of that section.

Note that the first line, Identified by, chapter title, and section text are all separated from one another by 2 `\n` characters.

**Currently, all of this information is recognized by our computers as a single file, or collection, of text.** If we want to be able to view laws, for example, by volume or by location, then we need to structure this text in a way that provides explicit information, such as dates, chapter names, etc., to our computers. To do this, **we'll create a table** that includes things like year, chapter, identified by, etc.--all information we can use in our analyses.

### Step 1: Getting the Text File

The text file for the corpus is located in the Carolina Digital Repository (CDR). In order to work with it, let's download it and add it to our Binder or local workspace in Jupyter Notebooks. You may remember that we used similar code when we first [gathered a selection of the corpus from the Internet Archive](02-GatheringACorpus.ipynb).

**To get the data's URL**, navigate to the [Jim Crow Laws](https://onthebooks.lib.unc.edu/laws/jim-crow-laws/) page on the *On The Books* website. Click on the link labeled "plain text format." This will take you to the CDR's page for the dataset. On the CDR page, *find the "Download the file" button. Right click on it, and select "Copy Link" (sometimes "Copy Link Location").*

Paste the URL in the script below over the text `'INSERT LINK HERE'`. That line should then look something like this:

`url = 'https://cdr.lib.unc.edu/downloads/6q182r84s?locale=en'`

**The following script will only work if you have completed the instructions above and added the link to the data file stored in the CDR.**

In [ ]:
# Requests helps us call up a webpage, or link to a file stored online,
# and access the content.
import requests

# Create a variable to hold the direct link to the text file.
# ADD THE LINK BETWEEN THE QUOTES BELOW.
url = 'ADD LINK HERE'

# Here's where we use the requests module to call up
# the content at the url we specified above.
r = requests.get(url)

# Create and open a new empty text file.
with open('on_the_books_text_jc_all.txt', 'wb') as f:
    
    # Write the contents of the online file into the new file.
    f.write(r.content)

# When finished, print the following:
print('Corpus downloaded.')

Now, look in the Jupyter Notebooks file browser to find the file.

### Step 2: From Text File to Dataframe

We'll begin by creating a **dataframe** (think of a dataframe as a table) using Pandas that can store the 4 main sections of data that we've identified. To do this, we'll split the text file into a list of laws and their accompanying data. For example, one item in this list will contain:

<img src="images/09-data-03.jpeg" width="90%" style="margin:20px; box-shadow: 25px 25px 20px -30px rgba(0, 0, 0);" alt="One law and its accompanying data from the corpus of North Carolina Jim Crow laws." title="One law and its accompanying data from the corpus of North Carolina Jim Crow laws." />

We'll then break this one law down into its own list so that each of the sections above becomes an item in that list:

<img src="images/09-data-04.jpeg" width="90%" style="margin:20px; box-shadow: 25px 25px 20px -30px rgba(0, 0, 0);" alt="Year/Volume, Law Type, Chapter, and Section of a Jim Crow law." title="Year/Volume, Law Type, Chapter, and Section of a Jim Crow law." />


<img src="images/09-data-05.jpeg" width="90%" style="margin:20px; box-shadow: 25px 25px 20px -30px rgba(0, 0, 0);" alt="Identified by field of a Jim Crow law." title="Identified by field of a Jim Crow law." />


<img src="images/09-data-06.jpeg" width="90%" style="margin:20px; box-shadow: 25px 25px 20px -30px rgba(0, 0, 0);" alt="Chapter title of a Jim Crow law." title="Chapter title of a Jim Crow law." />


<img src="images/09-data-07.jpeg" width="90%" style="margin:20px; box-shadow: 25px 25px 20px -30px rgba(0, 0, 0);" alt="Section text of a Jim Crow law." title="Section text of a Jim Crow law." />

We'll add each individual law's list to a compiled list of lists. Finally, we'll convert that list of lists into a spreadsheet (csv) file that we'll read back into a Pandas dataframe (table). Each row in the table will represent 1 law, and each column will hold one of the 4 parts above (Volume Law Type Chapter Section; Identified by; Chapter Title, and Section Text). It's OK if this doesn't fully make sense yet. Run the scripts below to create the dataframe, then go back and read through each step:

In [ ]:
# Import the regular expressions library.
# We'll need regex again to help us find the beginning and end 
# of each piece of data.
import re

# Open and read the plain text file of all identified Jim Crow laws.
with open("on_the_books_text_jc_all.txt", "r") as file:
    jclaws = file.read()
    
    # First, we'll split up the laws into a list with each law and 
    # its accompanying data as a separate list item. 
    # We'll use the 4 newline characters, "\n\n\n\n," to find
    # the beginning and end of each law.
    laws_split = jclaws.split("\n\n\n\n")
    
    # If you'd like to see what this list looks like, delete the "#"
    # next to "print" in the line below before (re)running this script:
    # print("List of laws:", laws_split)
    
    # Next, we'll create an empty list that we'll fill below 
    # with a list for each law of its for main parts including 
    # Volume Law Type Chapter Section, Identified by, Chapter Title, 
    # and Section Text.
    laws_list = []
    
    # For each law (list item) in laws_split, get the law's Section Text.
    for law in laws_split:
        
        # Convert the law from a list item to a string.
        law = str(law)
        
        # Split the law into its 4 main parts using the \n\n character 
        # pattern, and store these parts as a list.
        law_list = re.split("\n\n", law)
        
        # If you'd like to see each law in its new list form, delete
        # the "#" next to "print" in the line below before (re)running 
        # this script:
        # print(law_list)

        # Add the list of law parts above to the larger list that 
        # will contain all law lists, creating a list of lists.
        laws_list.append(law_list)

# If you'd like to see the list of lists, delete the "#" next to 
# "print" in the line below before (re)running this script:
print(laws_list)    

Now that we have our list of laws (or list of lists), we'll write these into a spreadsheet:

In [ ]:
# Import the csv module, which helps us create a csv (comma separated value) file.
import csv 

# Create the column headers for the file.
column_headers = ["VolumeLawTypeChapterSection","IdentifiedBy","ChapterTitle","SectionText"]  

# Create a new csv file and open it.
with open('jc_laws_list.csv', 'w') as f: 

    # This variable creates a writer that will add our laws to the csv file.
    write = csv.writer(f) 
    
    # Add the column headers first so that they appear in the first row.
    write.writerow(column_headers) 
    
    # Add each of the laws from laws_list.
    write.writerows(laws_list) 

You can [view the new sheet here](jc_laws_list.csv).

We've added this extra step to ensure that the data gets structured correctly. Some laws contain formatting that interferes with our four column structure--creating more columns than we need.

Now we'll read this spreadsheet back in so that we can work with the data. 

In [ ]:
# Import the pandas library, which we'll use to structure our data.
import pandas as pd

df = pd.read_csv("jc_laws_list.csv", sep=",")

# Print a preview of the dataframe for us to view here.
df.head()

**Take a few moments to study the dataframe we've just created:** can you now see how the laws have been restructured from a text file into a table that separates different information into columns?

Note the numbers labeling each row of the dataframe on the left side. These are unique identifiers for each row. They don't relate to the chapter or section numbers, but they can help us differentiate between the laws.



### Step 3: Refining the Dataframe

We now have the laws in a table, but if we know we'll want to be able to analyze the laws by volume, by law type, or by some other feature, we'll want to add some more columns to our dataframe. Pandas lets us modify an existing dataframe *without* having to create a new one. 

We'll begin by splitting the "Volume Law Type Chapter Section" column into 4 separate columns. We'll use regular expressions again to help us get each piece of data. In Pandas, square brackets, `[ ]`, can be used to identify an existing or new column name. We'll create a new column for a piece of data using 1 line of code that looks like this:

`df["Volumes"] = df["VolumeLawTypeChapterSection"].str.extract("(\d\d\d\d\/\d\d|\d\d\d\d)", expand=True)`

There's a lot happening in that 1 line, so let's break it down, from left to right, into the following steps:

1. `df["Volumes"] =` - Create a variable with a new column name.


2. `df["VolumeLawTypeChapterSection"]` - Identify which *existing* column the new column should pull it data from. 


3. `.str.` - Read the characters in the *existing* column.


4. `.extract("(\d\d\d\d\/\d\d|\d\d\d\d)",` - Extract only the characters that match a given regular expression.


5. `expand=True` - Finally, specify that the data extracted needs to be added to an entirely new column.

In the following script, we repeat the above steps 4 times to create new columns for Volumes, Law Type, Chapter Number, and Section Number:

In [ ]:
# First, let's get the volume. 
# The expand argument ("expand=True") adds a new column to the 
# dataframe. Our regular expression ("(\d\d\d\d\/\d\d|\d\d\d\d)") 
# helps us get volumes that are written as either "1873/4" or "1873".
# The pipe ("|") in the regular expression represents "or".
df["Volume"] = df["VolumeLawTypeChapterSection"].str.extract("(\d\d\d\d\/\d\d|\d\d\d\d)", expand=True)

# Let's repeat the above process for law types.
# In this case, we can search for specific labels, "Private Laws,"
# "Public Laws," or "Session Laws." Again, the pipe ("|") symbolizes "OR": 
# We're telling Pandas to search for and extract 
# Private Laws OR Public Laws OR Session Laws.
df["LawType"] = df["VolumeLawTypeChapterSection"].str.extract("(Private Laws|Public Laws|Session Laws)", expand=True)

# Now let's get the chapter number. This regular expression begins 
# with the characters "Ch". "\." tells Python to look for a period 
# (".") rather than using the period to symbolize any character. 
# And because chapter numbers vary in the number of digits 
# (e.g. 6 or 24 or 139), we use "\d" to look for 1 digit and "+" 
# to tell Python to look for additional digits until there are no more.
df["ChapterNum."] = df["VolumeLawTypeChapterSection"].str.extract("Ch\. (\d+)", expand=True)

# Next, the section number. We use the same basic construction as we 
# did for chapter, except we replace "Ch" with "Sec".
df["SectionNum."] = df["VolumeLawTypeChapterSection"].str.extract("Sec\. (\d+)", expand=True)

# Let's preview the dataframe again to see how it's changed.
df.head()

**How has the dataframe changed?** Where are the new columns? What do they contain? 

Note that the original column, "Volume Law Type Chapter Section" has not been changed. We could remove it, but we might decide to come back to it later--we'll leave it as is for now.

**We can also modify the contents of a column if we need to.** In the `Identified By` column, each cell contains "Identified by:" in addition to its value, e.g. "model" or "expert." Because we have a column label, we don't need the repeating "Identified by:" in each cell. Let's remove it:

In [ ]:
# In this line, we'll use "lstrip" to remove "Identified by: " from
# the IdentifiedBy column. Essentially, Python is overwriting the 
# existing column with the "model" or "expert" values without including
# "Identified by: ".
df["IdentifiedBy"] = df["IdentifiedBy"].str.lstrip("Identified by: ")

# Show us a preview of the updated dataframe.
df.head()

We can break down the line of code we just ran, from left to right, as follows:

`df["Identified By"] =` asks Python to look at the "Identified By" column and make the following changes to it.

`df["Identified By"].str` reads the contents of the "Identified By" column.

`.lstrip("Identified by: ")` looks at the beginning of each cell for "Identified by: " and removes, or "strips," it.

**Finally, let's add one more column of data: the number of words in each law.**

In [ ]:
df["WordCount"] = df["SectionText"].apply(lambda x: len(str(x).split(" ")))

# Show us a preview of the updated dataframe.
df.head()

**How did we do that? Let's break it down:**

`df["Word Count"] =` creates a new column name.

`df["Section Text"].apply` tells Python to *apply* whatever instructions come after this part of the line to the entire Section Text column.

`(lambda x: len(str(x).split(" ")))` in a nutshell counts the number of words in each Section Text cell. It's using a number of functions to do that, though, so let's break things down a little further:

- `lambda x:` -- `lambda` in Python is what's known as an "anonymous function" -- basically, it stores a given set of instructions and uses them on the value `x`. (Learn more about <a href="https://www.w3schools.com/python/python_lambda.asp" target="blank">lambda</a>.)


- `len()` gets the length of a string of characters. For example, `len("Hello World!")` would return the value `12`. Here, though, we want to use it to count *words*, not just characters.


- `str(x).` tells Python to read the contents of a cell, `x`, as a string (plain text).


- `split(" ")` tells Python to split that string by spaces, which creates a list of words. Punctuation is likely attached to some words, but because we're not counting individual word *length*, this doesn't matter. So `labor` and `labor,` both count as 1 word.

Working backwards, then, the `len()` function counts the number of *words* in the list created by `str(x).split(" ")`. `lambda x:` stores the instructions `len(str(x).split(" "))` to be used as many times as needed, and `apply` applies the instructions stored in `lambda x:` to every cell in the "Section Text" column. The word count for each Section is then stored in a cell on the *same row* in a new column called "Word Count".

Before we move on to the next and final module, let's save our dataframe as a .csv file so that we can access it:

In [ ]:
# Let's write this dataframe in a csv file. 
# We'll use a pipe ("|") to separate cells for now to avoid interfering with 
# comma usage in the Section Text and Chapter Title columns.
# We'll also set index to False, which will exclude the row numbers above
# from the csv file.
df.to_csv('jclaws_dataset.csv', sep="|", index=False)

# When the file has been created, print
print("jclaws_dataset.csv created.")

Locate the [jclaws_dataset.csv file](jclaws_dataset.csv) in the same folder as this tutorial and open it in a text editor (such as Notepad or TextEdit). Note how the pipes "|" separate each cell, or each piece of data.

<div class="alert alert-block alert-success">
    <strong>Review &amp; Next Steps:</strong> 
    <p>In this module, we learned some basic methods to correct OCR errors and change OCR'ed text using regular expressions. We also began structuring OCR'ed text as data using Pandas. In the next module, we'll see how Pandas can be used to not only structure data but also to perform exploratory analyses.</p>
</div>

## Resources <a class="anchor" id="resources"></a>

### Regular Expressions

- Knox, Doug. ["Understanding Regular Expressions"](https://programminghistorian.org/en/lessons/understanding-regular-expressions). *Programming Historian*.

- Lovett, M. ["Regular Expressions: The Complete Tutorial"](https://www.princeton.edu/~mlovett/reference/Regular-Expressions.pdf). Princeton University.

- O'Hara, Laura Turner. ["Cleaning OCR'ed Text with Regular Expressions"](https://programminghistorian.org/en/lessons/cleaning-ocrd-text-with-regular-expressions). *Programming Historian*.

- Yang, Alex. ["Python Regex Cheat Sheet"](https://www.dataquest.io/blog/regex-cheatsheet/). *Dataquest.*

### Pandas

- Burns, Halle. ["Crowdsourced-Data Normalization with Python and Pandas"](https://programminghistorian.org/en/lessons/crowdsourced-data-normalization-with-pandas#exploring-the-nypl-historical-menu-dataset). *Programming Historian*.

- ["Getting Started Tutorials"](https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html). Pandas.pydata.org.

- ["Pandas](https://unc-libraries-data.github.io/Python/Jupyter/Pandas.html). UNC Libraries Data.

- ["Pandas Tutorial"](https://www.w3schools.com/python/pandas/default.asp). W3Schools.

*This module is licensed under the [GNU General Public License v3.0](https://github.com/UNC-Libraries-data/OnTheBooks/blob/master/LICENSE). Individual images and data files associated with this module may be subject to a different license. If so, we indicate this in the module text.*